<a href="https://colab.research.google.com/github/NoVA0123/Anemia-Analysis/blob/main/Anemia_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the libraries

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle


#Importing the data

In [2]:
df = pd.read_excel('raw_file.xlsx')
df = df.drop(['Anemia level.1', 'Unnamed: 0'], axis = 1)

In [3]:
df.head()

,Age in 5-year groups,Type of place of residence,Highest educational level,Wealth index combined,Births in last five years,Age of respondent at 1st birth,Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal),Anemia level,Have mosquito bed net for sleeping (from household questionnaire),Smokes cigarettes,Current marital status,Currently residing with husband/partner,When child put to breast,Had fever in last two weeks,Hemoglobin level adjusted for altitude (g/dl - 1 decimal),"Taking iron pills, sprinkles or syrup"
0,25-29,Urban,Secondary,Richest,1,25,95,Moderate,Yes,No,Married,Living with her,105.0,No,114,No
1,35-39,Urban,Secondary,Richest,2,32,121,Not anemic,Yes,No,Married,Living with her,Immediately,No,102,Yes
2,20-24,Urban,Secondary,Richest,1,19,108,Moderate,Yes,No,Married,Living with her,Immediately,No,113,Yes
3,25-29,Urban,Higher,Richest,1,24,116,Mild,Yes,No,Married,Living with her,Days: 1,No,109,No
4,20-24,Urban,Higher,Richest,2,19,118,Mild,Yes,No,Married,Living with her,Immediately,No,96,Yes


Creating a function to transform categorical columns into numbers and saving categorical column names into a variable.

In [4]:
def categorical_transformer(basis):

    df[basis] = pd.factorize(df[basis])[0]

columns = df.columns.tolist()
del columns[4]
del columns[4]
del columns[4]
del columns[9]
del columns[10]

In [5]:
for x in columns:

    categorical_transformer(x)

# Few columns are transformed as follows

# Age in 5-year group:

1. 25-29: 0
2. 35-39: 1
3. 20-24: 2
4. 30-34: 3
5. 15-19: 4
6. 45-49: 5
7. 40-44: 6

# Highest level of education

1. Secondary: 0
2. Higher: 1
3. No education: 2
4. Primary: 3

# Wealth index

1. Richest: 0
2. Richer: 1
3. Middle: 2
4. Poorer: 3
5. Poorest: 4

In [6]:
df.head()

,Age in 5-year groups,Type of place of residence,Highest educational level,Wealth index combined,Births in last five years,Age of respondent at 1st birth,Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal),Anemia level,Have mosquito bed net for sleeping (from household questionnaire),Smokes cigarettes,Current marital status,Currently residing with husband/partner,When child put to breast,Had fever in last two weeks,Hemoglobin level adjusted for altitude (g/dl - 1 decimal),"Taking iron pills, sprinkles or syrup"
0,0,0,0,0,1,25,95,0,0,0,0,0,105.0,0,114,0
1,1,0,0,0,2,32,121,1,0,0,0,0,Immediately,0,102,1
2,2,0,0,0,1,19,108,0,0,0,0,0,Immediately,0,113,1
3,0,0,1,0,1,24,116,2,0,0,0,0,Days: 1,0,109,0
4,2,0,1,0,2,19,118,2,0,0,0,0,Immediately,0,96,1


Transforming 'When child put to breast' column into relevant numbers.

In [7]:
unique_val = df['When child put to breast'].unique()

def col_trans(values):

    if values == 'Immediately':

        return 0.0

    elif values == 'Days: 1':

        return 1.0

    elif values == 'Hours: 1':

        return 1/24

    else:

        return float(values)


df['When child put to breast'] = df['When child put to breast'].apply(col_trans)
df.head()

,Age in 5-year groups,Type of place of residence,Highest educational level,Wealth index combined,Births in last five years,Age of respondent at 1st birth,Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal),Anemia level,Have mosquito bed net for sleeping (from household questionnaire),Smokes cigarettes,Current marital status,Currently residing with husband/partner,When child put to breast,Had fever in last two weeks,Hemoglobin level adjusted for altitude (g/dl - 1 decimal),"Taking iron pills, sprinkles or syrup"
0,0,0,0,0,1,25,95,0,0,0,0,0,105.0,0,114,0
1,1,0,0,0,2,32,121,1,0,0,0,0,0.0,0,102,1
2,2,0,0,0,1,19,108,0,0,0,0,0,0.0,0,113,1
3,0,0,1,0,1,24,116,2,0,0,0,0,1.0,0,109,0
4,2,0,1,0,2,19,118,2,0,0,0,0,0.0,0,96,1


Preparing data for machinel learning.

I have used Standard scaler for scaling the data and train_test_split for splitting the data.

In [8]:
X = df.drop('Anemia level', axis = 1)
y = df['Anemia level']

scaler = StandardScaler()
X = scaler.fit_transform(X)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 20)

I have used XGBClassifier(extreme gradient boosting) from xgbosst libraray for classification.

In [9]:
classifier = XGBClassifier(n_estimators = 50, random_state = 2)
classifier.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

Predicting the test data and validating the accuracy.

In [11]:
predictions = classifier.predict(x_test)
print(accuracy_score(y_test, predictions)*100)

89.79591836734694


Saving the model.

In [13]:
pickle.dump(classifier, open('model.pkl', 'wb'))